# Project 2 - Medical Imaging 

## Imports

In [93]:
import os 
import re 

import numpy as np 
import pandas as pd 
from PIL import Image 
from matplotlib import pyplot as plt 
import seaborn as sns 

from skimage import morphology 
from skimage.transform import rotate 

from sklearn.preprocessing import StandardScaler 

from sklearn.feature_selection import mutual_info_classif, SelectKBest

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score 

from time import time 

## Global Variables

In [94]:
# Initial parameters for running the notebook 
PREPROCESS = False
COMPUTE_FEATURES = False 

In [95]:
# Paths to directories used 

ROOT_DIR = "../"
DATA_DIR = ROOT_DIR + "data"
# IM_DIR = DATA_DIR + "/ISIC-2017_Training_Data"
# MASK_DIR = DATA_DIR + "/ISIC-2017_Training_Part1_GroundTruth"
IM_DIR = DATA_DIR + "/filtered_images"
MASK_DIR = DATA_DIR + "/filtered_masks"
DIAGNOSIS_PATH = DATA_DIR + "/ISIC-2017_Training_Part3_GroundTruth.csv"


In [96]:
# Load all filenames into a dictionary
FILENAMES = {}
FILENAMES['images'] = sorted([IM_DIR + "/" + i for i in list(os.walk(IM_DIR))[0][2]])
FILENAMES['masks'] = sorted([f"{MASK_DIR}/{i}" for i in list(os.walk(MASK_DIR))[0][2]])
FILENAMES['image_num'] = len(FILENAMES['images'])

## Helper functions 

In [97]:
def make_sides_even(image): 
    '''
    Function to make the numbers of columns and rows in 
    an image even. 
    Input: An image 
    Output: An image
    '''
    # Convert image to numpy array
    image = np.array(image)
    
    # Check if the number of rows is even 
    if image.shape[0] % 2 != 0: 
        # Delete the first row
        image = np.delete(image,0,axis = 0)
    
    # Check if the number of columns is even 
    if image.shape[1] % 2 != 0: 
        # Delete the first column
        image = np.delete(image,0,axis = 1)
    
    # Convert numpy array back to image 
    image = Image.fromarray(image)
    # Return the updated image 
    return image

In [98]:
def filter_and_crop_image(image_path,mask_path): 
    '''
    Function to filter filter an image based on a given mask 
    and crop both image and mask to the relevant area. 
    '''
    # Instantiate both image and mask 
    image = Image.open(image_path) 
    mask = Image.open(mask_path)
    
    # Crop both image and mask to the bounding box
    # of the mask. 
    image_crop = image.crop(mask.getbbox())
    mask_crop = mask.crop(mask.getbbox())
    
    # Make the length and height of the the image and 
    # mask even 
    image_crop = make_sides_even(image_crop)
    mask_crop = make_sides_even(mask_crop)

    # Instantiate a blank image for a composite image
    tmp_image = Image.new("RGB",image_crop.size, 0)

    # Create a composite image based on the blank image, 
    # the cropped image and the mask 
    filtered_image = Image.composite(image_crop,tmp_image,mask_crop)
    
    # Return the filtered image, the cropped mask and the cropped image 
    return filtered_image, mask_crop, image_crop

## Preprocessing

In [99]:
# Check if the PREPROCESS variable is True 
if PREPROCESS: 
    # Try to create directories for the filtered images, cropped masks 
    # and cropped images 
    try: 
        os.makedirs(DATA_DIR + "/filtered_images")
        os.makedirs(DATA_DIR + "/filtered_masks")
        os.makedirs(DATA_DIR + "/cropped_images")

    except FileExistsError:
        print("Directories exist")
    except IsADirectoryError: 
        print("Directories exist")

    # Instantiate variables containing the paths to the filtered images, 
    # cropped masks and cropped images 
    IMAGE_FILTER_DIR = DATA_DIR + "/filtered_images"
    MASK_FILTER_DIR = DATA_DIR + "/filtered_masks"
    IMAGE_CROP_DIR = DATA_DIR + "/cropped_images"

    # Loop over all images and masks based on the number of images
    for i in range(FILENAMES['image_num']):
        # Instantiate variables with the paths to the currently processing image
        # and mask 
        mask_path = FILENAMES['masks'][i]
        image_path = FILENAMES['images'][i]

        # Filter and crop the image and mask 
        image, mask, crop = filter_and_crop_image(image_path,mask_path)

        # Instantiate variables containing the image and mask name 
        image_name = image_path.split("/")[-1].split(".")[-2]
        mask_name = mask_path.split("/")[-1].split(".")[-2]

        # Save the generated images and mask to files 
        image.save(IMAGE_FILTER_DIR + "/" + image_name + ".jpg")
        mask.save(MASK_FILTER_DIR + "/" + mask_name + ".png")
        crop.save(IMAGE_CROP_DIR + "/" + image_name + ".jpg")

        # Clear the temporary variables 
        del mask_path, image_path, image, mask, crop, image_name, mask_name
    
    # Load all the new filenames into the FILENAMES dictionary
    FILENAMES['filtered_images'] = sorted([f"{IMAGE_FILTER_DIR}/{i}" for i in list(os.walk(IMAGE_FILTER_DIR))[0][2]])
    FILENAMES['cropped_images'] = sorted([f"{IMAGE_CROP_DIR}/{i}" for i in list(os.walk(IMAGE_CROP_DIR))[0][2]])
    FILENAMES['cropped_masks'] = sorted([f"{MASK_FILTER_DIR}/{i}" for i in list(os.walk(MASK_FILTER_DIR))[0][2]])
else: 
    # Instantiate variables containing the paths to the filtered images, 
    # cropped masks and cropped images 
    IMAGE_FILTER_DIR = DATA_DIR + "/filtered_images"
    MASK_FILTER_DIR = DATA_DIR + "/filtered_masks"
    IMAGE_CROP_DIR = DATA_DIR + "/cropped_images"

    # Load all the new filenames into the FILENAMES dictionary
    FILENAMES['filtered_images'] = sorted([f"{IMAGE_FILTER_DIR}/{i}" for i in list(os.walk(IMAGE_FILTER_DIR))[0][2]])
    FILENAMES['cropped_images'] = sorted([f"{IMAGE_CROP_DIR}/{i}" for i in list(os.walk(IMAGE_CROP_DIR))[0][2]])
    FILENAMES['cropped_masks'] = sorted([f"{MASK_FILTER_DIR}/{i}" for i in list(os.walk(MASK_FILTER_DIR))[0][2]])


## Feature Extraction Functions

In [100]:
## Find the area and perimeter of the mask 
def get_area_perimeter(mask, erosion=2): 
    '''
    Function which takes in a mask for a
    given image and returns the area and 
    perimeter of the mask. 
    '''

    # Convert the mask to a Numpy array containing 1's and 0's
    mask = np.where(np.array(mask) ==255,1,0)

    # Calculate the area of the mask as the sum of the Numpy array
    area = np.sum(mask)
    
    # Erode the edge of the mask in order to calculate the perimeter 
    mask_erosion = morphology.binary_erosion(mask,morphology.disk(erosion))
        
    # Calculate the perimeter 
    perimeter = np.sum(mask - mask_erosion)
    # Return the area and the perimeter 
    return area, perimeter

In [101]:
def get_compactness(mask): 
    '''
    Function which takes in a mask for a given 
    image, calls the get_area_perimeter function
    to get the area and perimeter, and returns a 
    compactness score based upon [CITATION NEEDED]. 
    '''
    # Calculate the area and the perimeter 
    area, perimeter = get_area_perimeter(mask) 
    # Calculate the compactness 
    compactness = perimeter ** 2 / (4 * np.pi * area)
    # Return the compactness 
    return compactness 

In [102]:
def get_asymmetry(mask, rotation=45): 
    '''
    Takes in a mask for a given image, rotates it 
    180 times by one degree, compares the left and 
    right half and returns an average asymmetry score.
    '''
    # Instantiate the mask as a Numpy array 
    mask = np.array(mask)
    axes = 0 
    # Get the length and width of the mask 
    height, width = mask.shape 
    
    # Get the area of the lesion 
    lesion_area = len(np.where(mask != 0)[0])

    # Instantiate a list to holds the calculated differences  
    diffs = []

    while axes * rotation < 180:
        temp_mask = rotate(mask, axes * rotation)
        # length_lesion = np.nonzero(np.sum(temp_mask, axis = 0))[0][-1] - np.nonzero(np.sum(temp_mask, axis = 0))[0][0]

        left_mask = temp_mask[:, :int(width/2)] 
        right_mask = temp_mask[:, int(width/2):]
        right_mask = np.fliplr(right_mask)
        rotation_diff = np.where(left_mask != right_mask)
    
        diffs.append(len(rotation_diff[0]))
        
        axes += 1
    
    diff = np.mean(diffs)
    
    return diff / lesion_area 



In [103]:
def get_average_luminance(image): 
    '''
    A function which takes in an image and
    returns the average luminance. 
    Input: A PIL Image 
    Output: Average luminance of the image
    '''
    # Convert the image to grayscale and then to a Numpy array 
    grayscale = np.array(image.convert('L'))

    # Calculate the mean of the luminance
    average_luminance = round(np.mean(grayscale[grayscale > 0]))

    # Return the average luminance 
    return average_luminance

In [104]:
def get_luminance_variability(image,measure="variance"): 

    # Convert the image to grayscale and then to a Numpy array 
    grayscale = np.array(image.convert('L'))

    # Check if measure is 'variance'
    if measure == 'variance': 
        # Using numpy's variance method, return the variance in luminance
        return round(np.var(grayscale[grayscale > 0]))
    # Check if measure is 'standard_deviation'
    elif measure == "standard_deviation": 
        # Using numpy's standard deviation method, return the standard deviation
        return round(np.std(grayscale[grayscale > 0]))
    else: 
        # If measure is neither 'variance' nor 'standard_deviation', raise a 
        # ValueError. 
        raise ValueError("Only 'variance' or 'standard_deviation' accepted.") 

In [105]:
def get_avg_color(image): 
    '''
    A function which takes in an image and returns 
    the average color of the image. 
    '''
    # Split the image into separate color channels 
    r, g, b = image.split()
    # Instantiate a numpy array based on each color channel 
    r = np.array(r)
    g = np.array(g)
    b = np.array(b)
    # Calculate the mean of each color channel 
    average_color = (
        round(np.mean(r[r > 0])), 
        round(np.mean(g[g > 0])), 
        round(np.mean(b[b > 0]))
    )
    # Return a tuple of the average color 
    return average_color

In [106]:
def get_color_variance(image,measure="variance"): 
    '''
    A function which takes in an image and 
    returns the variance of the color. 
    '''
    # Split the image into separate color channels 
    r, g, b = image.split()
    # Instantiate a numpy array based on each color channel 
    r = np.array(r)
    g = np.array(g)
    b = np.array(b)
    # Check if measure is 'variance'
    if measure == "variance": 
        # Using numpy's variance method, calculate the variance of each color
        rgb = (
            np.var(r[r>0]),
            np.var(g[g>0]),
            np.var(b[b>0])
        )
    # Check if measure is 'standard_deviation'
    elif measure == "standard_deviation": 
        # Using numpy's standard deviation method, calculate the standard deviation
        # of each color 
        rgb = (
            np.std(r[r>0]),
            np.std(g[g>0]),
            np.std(b[b>0])
        )
    else: 
        # If measure is neither 'variance' nor 'standard_deviation', raise 
        # a ValueError 
        raise ValueError("Only 'variance' or 'standard_deviation' accepted.") 
    # Return the mean of the variances or standard deviation of the 3 colors
    return np.mean(rgb)

## Feature Extraction 

In [107]:
# Check if the COMPUTE_FEATURES variable is True 
if COMPUTE_FEATURES: 
    #Instantiate a dictionary to store the features and image_id's 
    feature_dictionary = {
        "image_id": [],
        "area": [], 
        "perimeter": [],
        "compactness": [], 
        "asymmetry": [], 
        "luminance_average": [],
        "luminance_variance": [],
        "red_average": [],
        "green_average": [],
        "blue_average": [],
        "color_variance": [],
    }

    # Loop over all filtered images and cropped masks 
    for i in range(FILENAMES['image_num']): 
        # Instantiate variables containing the paths to the image and mask
        filtered_image_path = FILENAMES['filtered_images'][i]
        cropped_mask_path = FILENAMES['cropped_masks'][i]

        # Get the image name from the image path 
        image_name = filtered_image_path.split("/")[-1].split(".")[-2]

        print(f"Currently working on {i} - Image id: {image_name}")

        # Open both the image and mask as PIL Images 
        filtered_image = Image.open(filtered_image_path)
        cropped_mask = Image.open(cropped_mask_path)

        # Calculate all features and append them to the relevant list in 
        # the feature_dictionary 
        feature_dictionary['image_id'].append(image_name)
        area, perimeter = get_area_perimeter(cropped_mask)
        feature_dictionary['area'].append(area)
        feature_dictionary['perimeter'].append(perimeter)
        feature_dictionary['compactness'].append(get_compactness(cropped_mask))
        feature_dictionary['asymmetry'].append(get_asymmetry(cropped_mask))
        feature_dictionary['luminance_average'].append(get_average_luminance(filtered_image))
        feature_dictionary['luminance_variance'].append(get_luminance_variability(filtered_image))
        red, green, blue = get_avg_color(filtered_image)
        feature_dictionary['red_average'].append(red)
        feature_dictionary['green_average'].append(green)
        feature_dictionary['blue_average'].append(blue)
        feature_dictionary['color_variance'].append(get_color_variance(filtered_image))
    # Instantiate a pandas DataFrame based on the feature_dictionary
    features = pd.DataFrame(feature_dictionary)
    # Write the features DataFrame to .csv 
    features.to_csv(ROOT_DIR + "/features/feature_set.csv",sep=";",index=False)
else: 
    # Read the previously calculated feature set into a pandas DataFrame 
    features = pd.read_csv(ROOT_DIR + "/features/feature_set.csv", sep=";", index_col=False)

    

## Feature Selection 

In [108]:
## Create list of features 
feature_list = list(features.columns)[1:]


## Load the Melanoma data 
data = pd.read_csv("../data/ISIC-2017_Training_Part3_GroundTruth.csv")

## Merge the Melanoma data with the feature data 
image_data = pd.merge(features, data, on = "image_id")



In [109]:
features

,image_id,area,perimeter,compactness,asymmetry,luminance_average,luminance_variance,red_average,green_average,blue_average,color_variance
0,ISIC_0000000,364956,3906,3.326704,0.169375,79,1617,85,77,82,1646.168808
1,ISIC_0000001,53426,1773,4.682254,0.075609,54,892,73,50,40,837.143114
2,ISIC_0000002,237402,4097,5.626492,0.127264,109,1210,126,104,107,1202.416805
3,ISIC_0000003,260708,3639,4.042033,0.090658,100,1506,130,93,68,1392.834772
4,ISIC_0000004,269476,3392,3.397674,0.118065,143,1562,183,125,150,1711.177114
...,...,...,...,...,...,...,...,...,...,...,...
1995,ISIC_0015220,21598298,36340,4.865645,0.109164,82,885,140,59,49,909.184420
1996,ISIC_0015233,7283040,18275,3.649154,0.126046,90,1887,119,80,63,1887.810925
1997,ISIC_0015260,5988478,21771,6.298403,0.155695,57,1230,80,50,34,1191.485062
1998,ISIC_0015284,11498389,24030,3.996324,0.102036,97,1989,137,84,66,2045.950121


In [110]:
# Create feature Dataframe and melanoma DataFrame
X = image_data[feature_list]
y = image_data['melanoma']

In [111]:
## Split into development data and test data 
X_dev, X_test, y_dev, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=1)

In [112]:
## Normalize using Scikit-Learn's StandardScaler 
scaler = StandardScaler().fit(X_dev)
X_dev = pd.DataFrame(scaler.transform(X_dev.values), columns=X_dev.columns)
X_test = pd.DataFrame(scaler.transform(X_test.values), columns=X_test.columns)


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [113]:
## Create Noise columns 
noise_cols = 4 
features_noise = feature_list + ['Noise' for _ in range(noise_cols)]
noise = np.random.RandomState(1).uniform(0,0.1, size = (noise_cols, X_dev.shape[0])).transpose()


# Merge noise columns with the feature columns 
X_select = np.hstack((X_dev,noise))
y_select = y_dev 


for k in range(1, 5+1): 
    kbest = SelectKBest(mutual_info_classif, k=k)
    kbest.fit(X_select, y_select)
    scores = kbest.scores_ 

    print(f"k = {k}\n")
    score_dict = {scores[i]: i for i in range(len(scores))}
    # print(sorted(score_dict))
    for val in sorted(score_dict, reverse=True)[:k]:
        # print(score_dict[val])
        print(features_noise[score_dict[val]])

k = 1

perimeter
k = 2

perimeter
asymmetry
k = 3

perimeter
asymmetry
area
k = 4

perimeter
asymmetry
red_average
area
k = 5

red_average
perimeter
asymmetry
area
Noise


## Exploring the addition of Age and Sex to the test set 


In [150]:
## Load the Melanoma data 
data = pd.read_csv("../data/ISIC-2017_Training_Part3_GroundTruth.csv")

## Load the Age and Sex data 

age_sex = pd.read_csv("../data/ISIC-2017_Training_Data_metadata.csv")


## Merge the Melanoma data with the feature data 
image_data = pd.merge(features, data, on = "image_id") 

## Merge the Age and Sex data with the merged 
merged_df = pd.merge(image_data, age_sex, on = 'image_id')


In [151]:
list(merged_df.columns)

['image_id',
 'area',
 'perimeter',
 'compactness',
 'asymmetry',
 'luminance_average',
 'luminance_variance',
 'red_average',
 'green_average',
 'blue_average',
 'color_variance',
 'melanoma',
 'seborrheic_keratosis',
 'age_approximate',
 'sex']

In [152]:
# Convert "unknown" to none  
merged_df['age_approximate'] = [i if i != "unknown" else None for i in merged_df['age_approximate']]

In [153]:
# Convert 'male' to -1, 'female' to 1 and 'unknown' to none 
merged_df['sex'] = [-1 if i == "male" else 1 if i == "female" else None for i in merged_df['sex']]

In [154]:
# Create feature list 
column_list = list(merged_df.columns)
feature_list = []
for i in column_list:
    if i != 'melanoma' and i != 'image_id' and i !=  'seborrheic_keratosis': 
        feature_list.append(i)

In [155]:
# Remove rows with null values 
merged_without_none = merged_df.dropna()

In [156]:
print(len(merged_without_none))

1744


In [157]:
# Create feature Dataframe and melanoma DataFrame
X = merged_without_none[feature_list]
y = merged_without_none['melanoma']

In [135]:
## Split into development data and test data 
X_dev, X_test, y_dev, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=1)

In [136]:
## Normalize using Scikit-Learn's StandardScaler 
scaler = StandardScaler().fit(X_dev)
X_dev = pd.DataFrame(scaler.transform(X_dev.values), columns=X_dev.columns)
X_test = pd.DataFrame(scaler.transform(X_test.values), columns=X_test.columns)

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [137]:
## Create Noise columns 
noise_cols = 4 
features_noise = feature_list + ['Noise' for _ in range(noise_cols)]
noise = np.random.RandomState(1).uniform(0,0.1, size = (noise_cols, X_dev.shape[0])).transpose()


# Merge noise columns with the feature columns 
X_select = np.hstack((X_dev,noise))
y_select = y_dev 


for k in range(1, 5+1): 
    kbest = SelectKBest(mutual_info_classif, k=k)
    kbest.fit(X_select, y_select)
    scores = kbest.scores_ 

    print(f"k = {k}\n")
    score_dict = {scores[i]: i for i in range(len(scores))}
    # print(sorted(score_dict))
    for val in sorted(score_dict, reverse=True)[:k]:
        # print(score_dict[val])
        print(features_noise[score_dict[val]])

k = 1

age_approximate
k = 2

age_approximate
perimeter
k = 3

age_approximate
perimeter
green_average
k = 4

age_approximate
sex
perimeter
area
k = 5

age_approximate
perimeter
area
Noise
Noise


## Generate Tables and Figures